In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df_total = pd.read_csv('C:/Users/krotenko.n/Documents/GitHub/New folder/data/df_total.csv', index_col=[0])
df_rozmitka = pd.read_csv('C:/Users/krotenko.n/Documents/GitHub/New folder/data/df_rozmitka.csv', index_col=[0])

df_fin = df_total.copy()
df_fin['extr7'] = df_rozmitka['extr7']

In [9]:
df_fin.loc[df_fin['extr7'], 'id_contract'] = range(18324)
df_fin['id_contract'] = df_fin['id_contract'].shift(-1).fillna(method='ffill')

In [10]:
df_fin['date'] = pd.to_datetime(df_fin['timestamp'], unit='s')
# df_fin['month'] =  df_fin['date'].dt.month
df_fin['day_of_week'] = df_fin['date'].dt.day_of_week
df_fin['second_of_day'] = df_fin['timestamp']%86400
df_fin.drop(columns=['date'], inplace=True)

In [11]:
df_fin['open_close_diff_perp'] = df_fin['open_perp'] - df_fin['close_perp']
df_fin['high_low_diff_perp'] = df_fin['high_perp'] - df_fin['low_perp']
df_fin['avg_trade_volume_perp'] = df_fin['volume_perp'] / df_fin['trades_perp']
df_fin['bid_ask_spread_abs_perp'] = df_fin['askPrice_perp'] - df_fin['bidPrice_perp']
df_fin['bid_ask_spread_perp'] = (df_fin['askPrice_perp'] - df_fin['bidPrice_perp'])/df_fin['askPrice_perp']
df_fin['bid_ask_qty_spread_perp'] = (df_fin['askQty_perp'] - df_fin['bidQty_perp'])/df_fin['askQty_perp']
df_fin['bid_ask_qty_spread_abs_perp'] = df_fin['askQty_perp'] - df_fin['bidQty_perp']
df_fin['bid_ask_qty_sum_perp'] = df_fin['askQty_perp'] + df_fin['bidQty_perp']


df_fin['open_close_diff_cq'] = df_fin['open_cq'] - df_fin['close_cq']
df_fin['high_low_diff_cq'] = df_fin['high_cq'] - df_fin['low_cq']
df_fin['avg_trade_volume_cq'] = df_fin['volume_cq'] / df_fin['trades_cq']
df_fin['avg_trade_volume_cq'].fillna(0, inplace=True)
df_fin['bid_ask_spread_abs_cq'] = df_fin['askPrice_cq'] - df_fin['bidPrice_cq']
df_fin['bid_ask_spread_cq'] = (df_fin['askPrice_cq'] - df_fin['bidPrice_cq'])/df_fin['askPrice_cq']
df_fin['bid_ask_qty_spread_cq'] = (df_fin['askQty_cq'] - df_fin['bidQty_cq'])/df_fin['askQty_cq']
df_fin['bid_ask_qty_spread_abs_cq'] = df_fin['askQty_cq'] - df_fin['bidQty_cq']


df_fin['perp_cq_open_spread_abs'] = df_fin['open_perp'] - df_fin['open_cq']
df_fin['perp_cq_open_spread'] = (df_fin['open_perp'] - df_fin['open_cq'])/df_fin['open_perp']


df_fin['short_term_expectations_v1'] = (df_fin['open_perp'] - df_fin['open_cq'])/(df_fin['time_left'] + 86400)
df_fin['short_term_expectations_v2'] = (df_fin['open_perp'] - df_fin['open_cq'])/(df_fin['time_left'] + 86400*9)
df_fin['short_term_expectations_v3'] = (df_fin['open_perp'] - df_fin['open_cq'])/(df_fin['time_left'] + 86400*90)
df_fin['short_term_expectations_v4'] = np.power(df_fin['open_perp'] - df_fin['open_cq'], 2)/(df_fin['time_left'] + 86400)
df_fin['short_term_expectations_v5'] = np.power(df_fin['open_perp'] - df_fin['open_cq'], 2)/(df_fin['time_left'] + 86400*9)
df_fin['short_term_expectations_v6'] = np.power(df_fin['open_perp'] - df_fin['open_cq'], 2)/(df_fin['time_left'] + 86400*90)
df_fin['short_term_expectations_v7'] = np.power(df_fin['open_perp'] - df_fin['open_cq'], 2)/np.sqrt(df_fin['time_left'] + 86400)
df_fin['short_term_expectations_v8'] = np.power(df_fin['open_perp'] - df_fin['open_cq'], 2)/np.sqrt(df_fin['time_left'] + 86400*9)
df_fin['short_term_expectations_v9'] = np.power(df_fin['open_perp'] - df_fin['open_cq'], 2)/np.sqrt(df_fin['time_left'] + 86400*90)
df_fin['short_term_expectations_v10'] = np.sqrt(np.abs(df_fin['open_perp'] - df_fin['open_cq']))/(df_fin['time_left'] + 86400)
df_fin['short_term_expectations_v11'] = np.sqrt(np.abs(df_fin['open_perp'] - df_fin['open_cq']))/(df_fin['time_left'] + 86400*9)
df_fin['short_term_expectations_v12'] = np.sqrt(np.abs(df_fin['open_perp'] - df_fin['open_cq']))/(df_fin['time_left'] + 86400*90)
df_fin['short_term_expectations_v13'] = np.sqrt(np.abs(df_fin['open_perp'] - df_fin['open_cq']))/np.sqrt(df_fin['time_left'] + 86400)
df_fin['short_term_expectations_v14'] = np.sqrt(np.abs(df_fin['open_perp'] - df_fin['open_cq']))/np.sqrt(df_fin['time_left'] + 86400*9)
df_fin['short_term_expectations_v15'] = np.sqrt(np.abs(df_fin['open_perp'] - df_fin['open_cq']))/np.sqrt(df_fin['time_left'] + 86400*90)



df_fin = pd.concat([df_fin, pd.get_dummies(df_fin['symbol_cq'], prefix='symbol',drop_first=True)], axis=1)
df_fin = pd.concat([df_fin, pd.get_dummies(df_fin['day_of_week'], prefix='dayofweek',drop_first=True)], axis=1)

In [21]:
def total_delta(series):
    return series.iloc[-1] - series.iloc[0]
def minmax_rel_diff(series):
    return (series.max() - series.min())/series.max()

features_groupped_by_contracts = df_fin.groupby('id_contract').agg({
    'open_perp' : ['min', 'max', total_delta, minmax_rel_diff],
    'open_cq' : ['min', 'max', total_delta, minmax_rel_diff],
    'volume_perp' : ['sum', 'mean'],
    'trades_perp' : ['sum', 'mean'],
    'volume_cq' : ['sum', 'mean'],
    'trades_cq' : ['sum', 'mean'],
    'bidPrice_perp' : ['min', 'max', total_delta, minmax_rel_diff],
    'askPrice_perp' : ['min', 'max', total_delta, minmax_rel_diff],
    'bidQty_perp' : ['sum', 'mean'],
    'askQty_perp' : ['sum', 'mean'],
    'bidPrice_cq' : ['min', 'max', total_delta, minmax_rel_diff],
    'askPrice_cq' : ['min', 'max', total_delta, minmax_rel_diff],
    'bidQty_cq' : ['sum', 'mean'],
    'askQty_cq' : ['sum', 'mean'],
    'time_left' : lambda x: (x.max()-x.min())
})
features_groupped_by_contracts.columns = features_groupped_by_contracts.columns.to_flat_index()
features_groupped_by_contracts.head()


,"(open_perp, min)","(open_perp, max)","(open_perp, total_delta)","(open_perp, minmax_rel_diff)","(open_cq, min)","(open_cq, max)","(open_cq, total_delta)","(open_cq, minmax_rel_diff)","(volume_perp, sum)","(volume_perp, mean)","(trades_perp, sum)","(trades_perp, mean)","(volume_cq, sum)","(volume_cq, mean)","(trades_cq, sum)","(trades_cq, mean)","(bidPrice_perp, min)","(bidPrice_perp, max)","(bidPrice_perp, total_delta)","(bidPrice_perp, minmax_rel_diff)","(askPrice_perp, min)","(askPrice_perp, max)","(askPrice_perp, total_delta)","(askPrice_perp, minmax_rel_diff)","(bidQty_perp, sum)","(bidQty_perp, mean)","(askQty_perp, sum)","(askQty_perp, mean)","(bidPrice_cq, min)","(bidPrice_cq, max)","(bidPrice_cq, total_delta)","(bidPrice_cq, minmax_rel_diff)","(askPrice_cq, min)","(askPrice_cq, max)","(askPrice_cq, total_delta)","(askPrice_cq, minmax_rel_diff)","(bidQty_cq, sum)","(bidQty_cq, mean)","(askQty_cq, sum)","(askQty_cq, mean)","(time_left, <lambda>)"
id_contract,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,21363.3,21445.0,-67.0,0.003810,21394.8,21479.4,-65.1,0.003939,5375.086,767.869429,36447,5206.714286,17.641,2.520143,323,46.142857,21365.2,21444.9,-67.4,0.003717,21365.3,21445.0,-69.0,0.003716,101.6230,14.517571,23.082,3.297429,21386.4,21476.7,-70.0,0.004205,21392.6,21478.3,-70.2,0.003990,0.703,0.100429,0.543,0.077571,360
1.0,21345.8,21364.6,8.9,0.000880,21375.7,21393.1,10.0,0.000813,3255.166,1085.055333,18755,6251.666667,9.248,3.082667,135,45.000000,21345.8,21367.0,11.3,0.000992,21345.9,21367.1,11.3,0.000992,28.1640,9.388000,13.551,4.517000,21371.6,21392.3,15.4,0.000968,21378.2,21400.0,14.7,0.001019,0.196,0.065333,0.066,0.022000,120
2.0,21332.2,21392.7,-42.6,0.002828,21361.1,21415.9,-38.5,0.002559,4455.528,318.252000,38030,2716.428571,13.511,0.965071,231,16.500000,21332.2,21392.7,-42.5,0.002828,21332.3,21392.8,-42.5,0.002828,137.9185,9.851321,180.339,12.881357,21356.5,21418.9,-46.6,0.002913,21361.5,21426.7,-46.4,0.003043,1.497,0.106929,3.626,0.259000,780
3.0,21324.0,21420.5,79.1,0.004505,21350.8,21448.2,90.6,0.004541,7508.049,341.274955,61165,2780.227273,21.816,0.991636,404,18.363636,21324.0,21420.4,79.1,0.004500,21324.1,21420.5,79.1,0.004500,274.9070,12.495773,183.247,8.329409,21347.7,21449.8,80.4,0.004760,21350.8,21455.8,87.4,0.004894,1.644,0.074727,1.834,0.083364,1260
4.0,21384.9,21435.1,-27.5,0.002342,21411.1,21467.8,-30.0,0.002641,1996.997,399.399400,16697,3339.400000,5.951,1.190200,134,26.800000,21384.9,21437.1,-27.5,0.002435,21385.0,21437.2,-27.5,0.002435,52.1610,10.432200,22.355,4.471000,21411.1,21465.7,-30.0,0.002544,21416.6,21477.6,-24.6,0.002840,0.141,0.028200,0.518,0.103600,240


In [22]:
print(features_groupped_by_contracts.isna().sum().sum())
df_fin.isna().sum()[df_fin.isna().sum()!=0]

0


diff           1
diff_shift1    1
id_contract    2
dtype: int64

In [23]:
df_fin.drop(columns=['timestamp', 'symbol_perp', 'symbol_cq', 'diff', 'diff_shift1'], inplace=True)
df_fin.dropna(inplace=True)